In [1]:
from landmark_detection.pipeline import Pipeline_Landmark_Detection
from benchmark.revisitop.dataset import configdataset
from benchmark.evaluation import run_evaluation2, save_evaluation_result

import os

In [2]:
VERSION = 0

force_rebuild = True
save_every = 500
config = {
    'version': VERSION,
    
    # DATABASE
    'min_area': 0.0,
    'min_sim_db': 0.0,
    
    # PIPELINE
    'detector_file': "yolov8n-oiv7.pt",
    'extractor_onnx_file': "cvnet-sg-v" + str(VERSION) + ".onnx",
    'pipeline_onnx_file': "pipeline-yolo-cvnet-sg-v" + str(VERSION) + ".onnx",
    'image_dim': (640, 640),
    'allowed_classes': [41,68,70,74,87,95,113,144,150,158,164,165,193,205,212,224,257,
                                  298,310,335,351,354,390,393,401,403,439,442,457,466,489,510,512,
                                  514,524,530,531,543,546,554,565,573,580,588,591],
    'score_thresh': 0.05,
    'iou_thresh': 0.25,
    'scales': [0.7071, 1.0, 1.4142],
    'mean': [0.485, 0.456, 0.406],
    'std': [0.229, 0.224, 0.225],
    'rgem_pr': 2.5,
    'rgem_size': 5,
    'gem_p': 4.6,
    'sgem_ps': 10.0,
    'sgem_infinity': False,
    'eps': 1e-8,
}
topk = 10
min_sim = 0.70
min_votes = 0.60
remove_inner_boxes = 0.5
join_boxes = True

In [12]:
# Fixed Paths
CLASS_NAMES_PATH = os.path.join("oiv7.yaml")
DATASETS_PATH = os.path.abspath("datasets")
ROXFORD5K_PATH = os.path.join(DATASETS_PATH, "roxford5k", "jpg")
ROXFORD5K_PKL = os.path.join(DATASETS_PATH, "roxford5k", "gnd_roxford5k.pkl")
ROXFORD5K_CSV = os.path.join(DATASETS_PATH, "roxford5k", "roxford5k_image_data.csv")
RESULTS_PATH = os.path.join("benchmark", "results", "results.json")

In [4]:
cfg = configdataset('roxford5k', DATASETS_PATH)

In [5]:
PIPELINE_CONFIG_PATH = os.path.join("configs", "pipeline-yolo-cvnet-sg-v" + str(VERSION))

# Dataset Oxford
ROXFORD5K_DF = os.path.join(DATASETS_PATH, "roxford5k", "results", "df_roxford5k-v" + str(VERSION) + ".pkl")
ROXFORD5K_DESC = os.path.join(DATASETS_PATH, "roxford5k", "results", "desc_roxford5k-v" + str(VERSION) + ".pkl")

In [6]:
pipeline = Pipeline_Landmark_Detection(
        detector_file = config['detector_file'],
        extractor_onnx_file = config['extractor_onnx_file'],
        pipeline_onnx_file = config['pipeline_onnx_file'],
        image_dim = config['image_dim'],
        allowed_classes = config['allowed_classes'],
        score_thresh = config['score_thresh'],
        iou_thresh = config['iou_thresh'],
        scales = config['scales'],
        mean = config['mean'],
        std = config['std'],
        rgem_pr = config['rgem_pr'],
        rgem_size = config['rgem_size'],
        gem_p = config['gem_p'],
        sgem_ps = config['sgem_ps'],
        sgem_infinity = config['sgem_infinity'],
        eps = config['eps'],
        topk = topk,
        min_sim = min_sim,
        min_votes = min_votes,
        remove_inner_boxes = remove_inner_boxes,
        join_boxes = join_boxes
)
pipeline.to_json(PIPELINE_CONFIG_PATH)

Creando versión ONNX del preprocess
Instanciando el preprocessor
Creando versión ONNX del detector
Ultralytics 8.3.146  Python-3.11.11 torch-2.7.0+cpu CPU (Intel Core(TM) i5-9300HF 2.40GHz)
YOLOv8n summary (fused): 72 layers, 3,492,527 parameters, 0 gradients, 10.5 GFLOPs

PyTorch: starting from 'C:\Users\rdiaz\Documents\GitHub\landmark-detection-with-retrieval\landmark_detection\models\yolov8n-oiv7.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 605, 8400) (6.9 MB)

ONNX: starting export with onnx 1.17.0 opset 16...
ONNX: slimming with onnxslim 0.1.54...
ONNX: export success  1.5s, saved as 'C:\Users\rdiaz\Documents\GitHub\landmark-detection-with-retrieval\landmark_detection\models\yolov8n-oiv7.onnx' (13.5 MB)

Export complete (2.1s)
Results saved to C:\Users\rdiaz\Documents\GitHub\landmark-detection-with-retrieval\landmark_detection\models
Predict:         yolo predict task=detect model=C:\Users\rdiaz\Documents\GitHub\landmark-detection-with-retrieval\landmark_dete

In [8]:
# Build/load roxford5k dataset
df_result_ox, descriptors_final_ox, places_db_ox = pipeline.build_image_database(
    image_folder = ROXFORD5K_PATH,
    df_pickle_path = ROXFORD5K_DF,
    descriptor_pickle_path = ROXFORD5K_DESC,
    return_places_db = True,
    force_rebuild = force_rebuild,
    save_every = save_every,
    min_area = config['min_area'],
    min_sim = config['min_sim_db']
)

Procesando imágenes: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5063/5063 [1:03:08<00:00,  1.34it/s]


In [9]:
# Evaluate roxford5k
roxford5k_eval = run_evaluation2(df_result_ox, places_db_ox, 'roxford5k')

>> roxford5k: Evaluating test dataset...
>> roxford5k: Loading features...
>> roxford5k: Retrieval...
>> roxford5k: mAP E: 75.49, M: 58.69, H: 30.34
>> roxford5k: mP@k[ 1  5 10] E: [      97.06       84.04       78.46], M: [      95.71       81.81       77.19], H: [      64.29       53.43       44.14]


In [10]:
# Evaluate roxford5k with bbox
roxford5k_eval_bbox = run_evaluation2(df_result_ox, places_db_ox, 'roxford5k', use_bbox=True)

>> roxford5k: Evaluating test dataset...
>> roxford5k: Loading features...
>> roxford5k: Retrieval...
>> roxford5k: mAP E: 70.87, M: 54.89, H: 26.25
>> roxford5k: mP@k[ 1  5 10] E: [      92.65       81.99       77.62], M: [         90       75.52       72.57], H: [      65.71       46.43       39.12]


In [15]:
roxford5k_eval

{'map_easy': np.float64(0.7548713229454185),
 'map_medium': np.float64(0.5868571475011939),
 'map_hard': np.float64(0.3034443151332889),
 'mpr_easy': array([    0.97059,     0.84044,     0.78456]),
 'mpr_medium': array([    0.95714,      0.8181,      0.7719]),
 'mpr_hard': array([    0.64286,     0.53429,     0.44143])}

In [14]:
save_evaluation_result(roxford5k_eval, roxford5k_eval_bbox, RESULTS_PATH, config)

TypeError: Object of type ndarray is not JSON serializable